In [1]:
!apt-get install poppler-utils
!pip install pdf2image

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.6 [186 kB]
Fetched 186 kB in 0s (448 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 125044 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.6_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.6) ...
Setting up poppler-utils (22.02.0-2ubuntu0.6) ...
Processing triggers for man-db (2.10.2-1) ...


In [2]:
!pip install PyMuPDF easyocr Pillow numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.

In [14]:
from pdf2image import convert_from_path
from PIL import Image
import easyocr
import numpy as np

def pdf_to_text(pdf_path):
    # Конвертировать PDF в изображения
    images = convert_from_path(pdf_path)

    reader = easyocr.Reader(['ru'], gpu=True)
    total_text = []

    # Обработать каждое изображение
    for img in images:
        # Преобразовать PIL.Image в формат, который понимает easyocr
        img_array = np.array(img)
        # Распознать текст на текущей странице
        results = reader.readtext(img_array)

        # Собрать текст из всех найденных блоков
        page_text = ' '.join([result[1] for result in results])
        total_text.append(page_text)

    return '\n'.join(total_text)

In [15]:
# Использование
result = pdf_to_text('/content/64497298726756.pdf')

# Сохранить результат в файл
with open('output.txt', 'w', encoding='utf-8') as f:
    f.write(result)

print("Результат сохранен в файл output.txt")

Результат сохранен в файл output.txt


In [16]:
from pdf2image import convert_from_path
from PIL import Image
import easyocr
import numpy as np
import pandas as pd
import re

def pdf_to_text(pdf_path):
    # Конвертировать PDF в изображения
    images = convert_from_path(pdf_path)

    reader = easyocr.Reader(['ru'], gpu=True)
    total_text = []

    # Обработать каждое изображение
    for img in images:
        # Преобразовать PIL.Image в формат, который понимает easyocr
        img_array = np.array(img)
        # Распознать текст на текущей странице
        results = reader.readtext(img_array)

        # Собрать текст из всех найденных блоков
        page_text = ' '.join([result[1] for result in results])
        total_text.append(page_text)

    return '\n'.join(total_text)

def parse_text_to_df(text):
    # Регулярные выражения для извлечения данных
    patterns = {
        'Номер постановления': r'ПОСТАНОВЛЕНИЕ (\d{20})',
        'Дата нарушения': r'(\d{2}\.\d{2}\.\d{4})',
        'Номер ТС': r'регистрационный знак (.*?),',
        'Номер ТС-2': r'регистрационного знака: (.*?) ',
        'Сумма штрафа': r'штрафа в размере (\d+)',
        'Номер СТС': r'СТС: (\d{10})'
    }

    # Извлечение данных
    data = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        data[key] = match.group(1) if match else None

    # Поиск времени нарушения
    time_patterns = [
        r'(\d{2}:\d{2}:\d{2})',  # Первый шаблон (часы:минуты:секунды)
        r'(\d{2}.\d{2}:\d{2})'   # Второй шаблон (часы.минуты:секунды)
    ]
    data['Время нарушения'] = None  # По умолчанию
    for pattern in time_patterns:
        match = re.search(pattern, text)
        if match:
            data['Время нарушения'] = match.group(1)
            break  # Прерываем цикл, если время найдено

    # Поиск адреса нарушения
    address_patterns = [
        r'по адресу (.*?) водитель',  # Первый шаблон
        r'по адресу (.*?) зафиксировано'  # Второй шаблон
    ]
    data['Адрес (местоположение) нарушения'] = None  # По умолчанию
    for pattern in address_patterns:
        match = re.search(pattern, text)
        if match:
            data['Адрес (местоположение) нарушения'] = match.group(1)
            break  # Прерываем цикл, если адрес найден

    return pd.DataFrame([data])

# Использование
pdf_path = '/content/64497298726756.pdf'
text = pdf_to_text(pdf_path)

# Создание датасета
df = parse_text_to_df(text)

# Сохранение результата в CSV
df.to_csv('output.csv', index=False, encoding='utf-8')

print("Результат сохранен в файл output.csv")
print(df)

Результат сохранен в файл output.csv
    Номер постановления Дата нарушения   Номер ТС Номер ТС-2 Сумма штрафа  \
0  18810555240809076631     09.08.2024  СЗ99ТХ716  СЗ99ТХ716         None   

    Номер СТС Время нарушения  \
0  9969997208        14.53:04   

                    Адрес (местоположение) нарушения  
0  ФАД Р-402 Тюмень-Омск 48бкм. Г Тюкалинск; Омск...  


In [12]:
import re
import pandas as pd
from pdf2image import convert_from_path
import easyocr
import numpy as np
import os  # Импорт модуля для работы с файловой системой

def pdf_to_text(pdf_path):
    # Конвертировать PDF в изображения
    images = convert_from_path(pdf_path)

    reader = easyocr.Reader(['ru'], gpu=True)
    total_text = []

    # Обработать каждое изображение
    for img in images:
        # Преобразовать PIL.Image в формат, который понимает easyocr
        img_array = np.array(img)
        # Распознать текст на текущей странице
        results = reader.readtext(img_array)

        # Собрать текст из всех найденных блоков
        page_text = ' '.join([result[1] for result in results])
        total_text.append(page_text)

    return '\n'.join(total_text)

# Регулярные выражения для извлечения данных
patterns = {
    'Номер постановления': r'ПОСТАНОВЛЕНИЕ (\d{20})',
    'Дата нарушения': r'(\d{2}\.\d{2}\.\d{4})',
    'Время нарушения': r'(\d{2}:\d{2}:\d{2})',
    'Адрес (местоположение) нарушения': r'по адресу (.*?) водитель',
    'Номер ТС': r'регистрационный знак (.*?),',  # Измененное регулярное выражение
    'Сумма штрафа': r'штрафа в размере (\d+)',
    'Номер СТС': r'СТС: (\d{10})'
}

# Создаем пустой список для хранения данных
all_data = []

# Папка с PDF-файлами
input_folder = '/content/drive/MyDrive/input'

# Проходим по всем файлам в папке input
for filename in os.listdir(input_folder):
    if filename.endswith('.pdf'):  # Проверяем, что файл имеет расширение .pdf
        pdf_path = os.path.join(input_folder, filename)  # Полный путь к файлу
        result = pdf_to_text(pdf_path)  # Извлекаем текст из PDF

        # Извлечение данных
        data = {'Номер файла': filename}  # Добавляем номер файла в данные
        for key, pattern in patterns.items():
            match = re.search(pattern, result)
            if match:
                data[key] = match.group(1)
            else:
                data[key] = None

        # Добавляем данные в общий список
        all_data.append(data)

# Создание датасета
df = pd.DataFrame(all_data)

# Сохранение результата в CSV
df.to_csv('output.csv', index=False, encoding='utf-8')

print("Результат сохранен в файл output.csv")

KeyboardInterrupt: 

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import re
import pandas as pd
from pdf2image import convert_from_path
import easyocr
import numpy as np
import os
import torch  # Для проверки доступности GPU

# Проверка доступности GPU
if torch.cuda.is_available():
    print("GPU доступен. Используется GPU для обработки.")
    gpu_enabled = True
else:
    print("GPU недоступен. Используется CPU.")
    gpu_enabled = False

def pdf_to_text(pdf_path):
    # Конвертировать PDF в изображения с использованием многопоточности
    images = convert_from_path(pdf_path, thread_count=10)

    # Инициализация EasyOCR с использованием GPU, если он доступен
    reader = easyocr.Reader(['ru'], gpu=gpu_enabled)
    total_text = []

    # Обработать каждое изображение
    for img in images:
        # Преобразовать PIL.Image в формат, который понимает easyocr
        img_array = np.array(img)
        # Распознать текст на текущей странице
        results = reader.readtext(img_array)

        # Собрать текст из всех найденных блоков
        page_text = ' '.join([result[1] for result in results])
        total_text.append(page_text)

    return '\n'.join(total_text)

def parse_text_to_df(text):
    # Регулярные выражения для извлечения данных
    patterns = {
        'Номер постановления': r'ПОСТАНОВЛЕНИЕ (\d{20})',
        'Дата нарушения': r'(\d{2}\.\d{2}\.\d{4})',
        'Номер ТС': r'регистрационный знак (.*?),',
        'Номер ТС-2': r'регистрационного знака: (.*?) ',
        'Сумма штрафа': r'размере (\d+) ',
        'Номер СТС': r'СТС: (\d{10})'
    }

    # Извлечение данных
    data = {}
    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        data[key] = match.group(1) if match else None

    # Поиск времени нарушения
    time_patterns = [
        r'(\d{2}:\d{2}:\d{2})',  # Первый шаблон (часы:минуты:секунды)
        r'(\d{2}.\d{2}:\d{2})'   # Второй шаблон (часы.минуты:секунды)
    ]
    data['Время нарушения'] = None  # По умолчанию
    for pattern in time_patterns:
        match = re.search(pattern, text)
        if match:
            data['Время нарушения'] = match.group(1)
            break  # Прерываем цикл, если время найдено

    # Поиск адреса нарушения
    address_patterns = [
        r'по адресу (.*?) водитель',  # Первый шаблон
        r'по адресу (.*?) зафиксировано'  # Второй шаблон
    ]
    data['Адрес (местоположение) нарушения'] = None  # По умолчанию
    for pattern in address_patterns:
        match = re.search(pattern, text)
        if match:
            data['Адрес (местоположение) нарушения'] = match.group(1)
            break  # Прерываем цикл, если адрес найден

    return data

# Создаем пустой список для хранения данных
all_data = []

# Папка с PDF-файлами
input_folder = '/content/drive/MyDrive/input'

# Проходим по всем файлам в папке input
for filename in os.listdir(input_folder):
    if filename.endswith('.pdf'):  # Проверяем, что файл имеет расширение .pdf
        pdf_path = os.path.join(input_folder, filename)  # Полный путь к файлу
        print(f"Обработка файла: {filename}")  # Логирование процесса
        result = pdf_to_text(pdf_path)  # Извлекаем текст из PDF

        # Извлечение данных с использованием функции parse_text_to_df
        data = parse_text_to_df(result)
        data['Номер файла'] = os.path.splitext(filename)[0]  # Добавляем номер файла без расширения

        # Добавляем данные в общий список
        all_data.append(data)

# Создание датасета
df = pd.DataFrame(all_data)

# Сохранение результата в CSV
df.to_csv('/content/drive/MyDrive/output.csv', index=False, encoding='utf-8')

print("Результат сохранен в файл output.csv")

GPU доступен. Используется GPU для обработки.
Обработка файла: 66604798269081.pdf
Обработка файла: 62095098682641.pdf
Обработка файла: 64497298648959.pdf
Обработка файла: 14588298307467.pdf
Обработка файла: 61491098658302.pdf
Обработка файла: 63097598717419.pdf
Обработка файла: 42094798304040.pdf
Обработка файла: 45081198559610.pdf
Обработка файла: 40097998856885.pdf
Обработка файла: 63097598827378.pdf
Обработка файла: 45081498078972.pdf
Обработка файла: 42094798177576.pdf
Обработка файла: 64497298648942.pdf
Обработка файла: 42094798227257.pdf
Обработка файла: 45081398949327.pdf
Обработка файла: 40097998837150.pdf
Обработка файла: 64497298101782.pdf
Обработка файла: 42094798212604.pdf
Обработка файла: 64497298698787.pdf
Обработка файла: 45081498019081.pdf
Обработка файла: 64497298678949.pdf
Обработка файла: 61094298144875.pdf
Обработка файла: 45081498125393.pdf
Обработка файла: 68092198406904.pdf
Обработка файла: 64497298726756.pdf
Обработка файла: 45081398959098.pdf
Обработка файла: 6